In [1]:
%%time
import pandas as pd

from target_encoding import TargetEncoderClassifier, TargetEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression

Wall time: 2.1 s


In [2]:
%%time
train=pd.read_csv("Xx_train.csv")
test=pd.read_csv("Xx_test.csv")
sample_submission = pd.read_csv('sample_submission.csv')

cv = StratifiedKFold(n_splits=100, shuffle=True, random_state=1000)

Wall time: 1.88 s


In [3]:
%%time
len_uniques = []
for c in train.columns.drop(['is_click']):
    le = LabelEncoder()
    le.fit(pd.concat([train[c], test[c]])) 
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])
    print(c, len(le.classes_))
    len_uniques.append(len(le.classes_))
    
X = train.drop(['impression_id','is_click'], axis=1)
y = train['is_click']

impression_id 328284
user_id 89157
app_code 522
os_version 3
is_4G 2
time 6
item_id 18523
device_type 3
session_id 43470
item_price 4933
category_1 15
category_2 75
category_3 314
product_type 3961
Wall time: 3.58 s


In [4]:
%%time
ALPHA = 75
MAX_UNIQUE = max(len_uniques)
FEATURES_COUNT = X.shape[1]

Wall time: 0 ns


In [5]:
%%time
'''
alpha: float or int, smoothing for generalization.

max_unique: int, maximum number of unique values in a feature. 
            If there are more unique values inside the feature,
            then the algorithm will split this feature into bins, 
            the number of max_unique.

used_features: int, this is a number of used features for prediction
               The algorithm encodes all features with the average value of the target, 
               then the std is considered inside each feature,
               and "used_features" features with the highest std are selected to use only informative features. 
'''

enc = TargetEncoderClassifier(alpha=ALPHA, max_unique=MAX_UNIQUE, used_features=FEATURES_COUNT)
score = cross_val_score(enc, X, y, scoring='roc_auc', cv=cv)
print(f'score: {score.mean():.4}, std: {score.std():.4}')

enc.fit(X, y)
pred_enc = enc.predict_proba(test.drop('impression_id', axis=1))[:,1]

score: 0.682, std: 0.02405
Wall time: 45min 58s


In [6]:
%%time
'''
split: list of int or cross-validator class,
            if split is [], then algorithm will encode features without cross-validation
            This situation features will overfit on target

            if split len is 1 for example [5], algorithm will encode features by using cross-validation on 5 folds
            This situation you will not overfit on tests, but when you will validate, your score will overfit

            if split len is 2 for example [5, 3], algorithm will separate data on 5 folds, afterwords
            will encode features by using cross-validation on 3 folds
            This situation is the best way to avoid overfit, but algorithm will use small data for encode.
'''


enc = TargetEncoder(alpha=ALPHA, max_unique=MAX_UNIQUE, split=[cv])
X_train = enc.transform_train(X=X, y=y)
X_test = enc.transform_test(test.drop('impression_id', axis=1))

Wall time: 42min 19s


In [7]:
%%time
lin = LogisticRegression()
score = cross_val_score(lin, X_train, y, scoring='roc_auc', cv=cv)
print(f'score: {score.mean():.4}, std: {score.std():.4}')


lin.fit(X_train, y)
pred_lin = lin.predict_proba(X_test)[:,1]

c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


score: 0.7349, std: 0.02333


c:\users\professionally_saket\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 2min 49s


In [8]:
%%time
sample_submission['is_click'] = pred_enc + pred_lin
sample_submission.to_csv('n_submission.csv', index=False)

Wall time: 754 ms
